<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_KoBERT_eval_cc_std_70000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글드라이브 연동
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [2]:
SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000_cc.pt'

In [3]:
# !ls -l /content/drive/MyDrive/KAIST/SEP531

#KoBERT 다운로드(사용)#

In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9wgagly5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9wgagly5
     |████████████████████████████████| 344 kB 4.2 MB/s 
     |████████████████████████████████| 46.9 MB 2.4 MB/s 
     |████████████████████████████████| 4.9 MB 42.5 MB/s 
     |████████████████████████████████| 1.2 MB 37.5 MB/s 
     |████████████████████████████████| 3.4 MB 39.3 MB/s 
     |████████████████████████████████| 61 kB 519 kB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=b28f8e1b9a99ab345a93a65c7860449fde94bb7eaa55532f1d80405e75c1a805
  Stored in directory: /tmp/pip-ephem-wheel-cache-njw1ndeu/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [9]:
!pip install -U torchtext==0.6.0
!pip install transformers

     |████████████████████████████████| 64 kB 1.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [10]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

#KoBERT 입력 데이터로 변환(사용)#

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [14]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [15]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [16]:
filename = '/content/drive/MyDrive/KAIST/SEP531/std_test_10500.csv'

In [17]:
import csv

texts = []
labels = []

with open(filename, 'r') as f:
  csv_data = csv.reader(f, delimiter='|')
  for i, line in enumerate(csv_data):
    #if(i == 10000):
    #  break
    print('[%d] %s' % (i, line))
    text = line[0]
    label = int(line[1])
    texts.append(text)
    labels.append(label)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[5500] ['근데 여자친구가 엄청 싫어해요.', '3']
[5501] ['스쿼트하는데 허벅지폭팔할거같아요', '3']
[5502] ['너무맘고생했어서잊고싶다다른사람으로', '3']
[5503] ['시리고 시린 겨울나무가 된 것 같아 고민0', '3']
[5504] ['어떻게하면 그렇게 보이거나 될 수 있을까요 ㅠㅠ', '3']
[5505] ['연락을  자주해도 눈앞에 없으니 힘드네요', '3']
[5506] ['현기증 난단 말이에여 ㅠㅠ0', '3']
[5507] [' 군생활.. 그것보다 힘든건 너와못보고 연락을못하는것.', '3']
[5508] ['살빼야겟죠...', '3']
[5509] ['일하고 싶은 곳이 있어도 전화를 하기가 좀처럼 힘이듭니다.', '3']
[5510] [' 그게 아니라면 삶을 이어가기가 너무 힘들 것 같아요.', '3']
[5511] ['1년동안 어떻게 참음 ? ㅠ', '3']
[5512] ['  하루 두세시간 자고 주말도 일하고...', '3']
[5513] ['아 힘들다0', '3']
[5514] ['맛있는게 뭐가 있을까..', '3']
[5515] ['선배 고무신분들 조언좀 해주세요 ㅜㅜ', '3']
[5516] ['그래서 제가 걱정되서 뭔일 잇냐고 근데 그건아니래요', '3']
[5517] ['미치겠다...', '3']
[5518] [' 머리아프고 기력없다', '3']
[5519] ['끊기가 힘들지', '3']
[5520] ['아이 스트레스며 혹여 잘못될까 걱정부터앞섶니다...', '3']
[5521] ['지금 너무 졸리고 어지러운데 어카죠', '3']
[5522] ['다름이아니라 이제 곧여자친구의 생일이다가오고 있습니다^^', '3']
[5523] [' 매번싸움..정말지칩니다...', '3']
[5524] ['신경성 위염 달고 사는데 미쳐버린다', '3']
[5525] ['소포보내면 안에서 선임들이 뺏어먹는다는데 정말 다먹어요?ㅠㅠㅠ', '3']
[5526] 

In [18]:
y = []
p = []
def Eval():
  for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    label_idx = label
    #label_idx = r_dic.get(label)
    idx, val = predict(text)
    #print('(%s/%s) %s\n' % (val, label, text))
    #if(i % 1000 == 0):
    print('[%d](%c)(%d/%d) %s\n' % (i, 'O' if (idx == label_idx) else 'X', idx, label_idx, text))
    p.append(idx)
    y.append(label_idx)

  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [19]:
Eval()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[8000](O)(5/5) 생일축하해요!

[8001](O)(5/5) 2일 후면 예디 팬 된지 딱 1년 되네요ㅎㅎ

[8002](O)(5/5)  우와 이번활동 많~이 수고했어요ㅠㅠ

[8003](O)(5/5)   오늘 너무 자랑스럽구요...

[8004](O)(5/5) 3월내내 맴버들 생일이 없어서 축하글 오랜만에 써요ㅠㅠ

[8005](O)(5/5)   다시한번 추카추카~~

[8006](O)(5/5) 유재석 Jr.의 탄생을 축하드려요~

[8007](O)(5/5) 이번 해투는 가요제이후로 절 젤로 웃긴, 도니님다운 몸개그와 언변이었다고 생각합니다!

[8008](O)(5/5)   백주부님  파이팅 입니다 ^^

[8009](X)(1/5)  나오신다길래 조아서 히히히 다보구 이름을 쳐봤는데 펜카페가나오드라구요^^!

[8010](O)(5/5) 언제어디서나 기분 좋게 바라 볼수 있는분!!

[8011](O)(5/5)  진심으로 생일축하한다

[8012](O)(5/5) 오늘밤 SBS 11시 25분 첫방송~!!!

[8013](O)(5/5)  생일축하해 찬아♡ 아!

[8014](O)(5/5)  김.진.선.도니오빠 참 조아요.///^-^

[8015](O)(5/5)  아시아투어 건강하게잘갔다와요!!

[8016](O)(5/5) 백선생님요리 정말제미있어요.

[8017](O)(5/5) 만능간장으로 채소요리도 쉽게~~

[8018](O)(5/5) 백주부님 덕분에 요리가 갈수록 스피드하고 맛이 있어지는거 같아요~^^*

[8019](O)(5/5) 유재석 오라버니 하늘이 파랗게 있으니 날씨가 좋더라고요

[8020](O)(5/5) 안병원선수의 LG팬으로써 진심으로 축하 드리구여.

[8021](O)(5/5)  꼭 본방사수해서 시청잘할께요^^

[8022](O)(5/5)  그리고 실검 1위도 축하해요????

[8023](O)(5/5) 나도 용빈이에요!!!

[8024](O)(5/5) 메.찾.사 를 개설한지 

#F1 score

In [20]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict



In [21]:

print(metrics.precision_score(y, p, average=None)) #
print(metrics.precision_score(y, p, average=None).mean()) #
print(metrics.precision_score(y, p, average='macro')) #1.0
print(metrics.precision_score(y, p, average='micro')) #1.0

[0.50619985 0.46717309 0.55306719 0.63963964 0.45543416 0.82335766
 0.39598997]
0.5486945091185149
0.5486945091185149
0.5399047619047619


In [22]:
# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))





accuracy 0.5399047619047619
precision1 [0.50619985 0.46717309 0.55306719 0.63963964 0.45543416 0.82335766
 0.39598997]
precision2 0.5486945091185149
precision3 0.5486945091185149
precision4 0.5399047619047619
recall1 [0.46266667 0.574      0.37866667 0.66266667 0.528      0.752
 0.42133333]
recall2 0.5399047619047618
recall3 0.5399047619047618
recall4 0.5399047619047619
f1_score1 [0.48345524 0.51510619 0.44954491 0.65094957 0.48903983 0.78606272
 0.40826873]
f1_score2 0.5403467431562754
f1_score3 0.5403467431562754
f1_score4 0.5399047619047619
              precision    recall  f1-score   support

           0       0.51      0.46      0.48      1500
           1       0.47      0.57      0.52      1500
           2       0.55      0.38      0.45      1500
           3       0.64      0.66      0.65      1500
           4       0.46      0.53      0.49      1500
           5       0.82      0.75      0.79      1500
           6       0.40      0.42      0.41      1500

    accuracy    

In [23]:
print("\nLogistic Regression")
# assuming your already have a list of actual_class and predicted_class from the logistic regression classifier
lr_roc_auc_multiclass = roc_auc_score_multiclass(y, p)
print(lr_roc_auc_multiclass)

# Sample output
# Logistic Regression
# {0: 0.5087457159427196, 1: 0.5, 2: 0.5, 3: 0.5114706737345112, 4: 0.5192307692307693}
# 0.5078894317816


Logistic Regression
{0: 0.6937222222222222, 1: 0.7324444444444443, 2: 0.6638333333333333, 3: 0.8002222222222222, 4: 0.7113888888888888, 5: 0.8625555555555556, 6: 0.6571111111111111}
